In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

In [3]:
from openslide import OpenSlide

In [4]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [5]:
# Python < 3.8?
import sys
#! pip install pickle5

if sys.version_info.minor < 8:
    import pickle5 as pickle

In [6]:
import sys
sys.path.append("../..")
from registration_tree import Rect, QuadTree

In [7]:
slide_list = {path.name: path for path in Path(r"D:\Datasets\ColonCancer\SingleSlide").glob("*/*.*") 
                      if path.suffix in [".tif"]}

slide_list

{'CRC-AI-11 F X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-11 F X40.tif'),
 'CRC-AI-14 1G X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-14 1G X40.tif'),
 'CRC-AI-15 X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-15 X40.tif'),
 'CRC-AI-8 E X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-8 E X40.tif'),
 'CRC-AI-8 H X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-8 H X40.tif'),
 'CRC-AI-11 F X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-11 F X40 IHC.tif'),
 'CRC-AI-14 1G X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-14 1G X40 IHC.tif'),
 'CRC-AI-15 X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-15 X40 IHC.tif'),
 'CRC-AI-8 E X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-8 E X40 IHC.tif'),
 'CRC-AI-8 H X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC

In [8]:
configuration = Configuration()
configuration.username = 'exact' #'exact'
configuration.password = 'exact' #'exact'
configuration.host = "http://127.0.0.1:1337" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [9]:
annotation_types_points = {}

for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.id] = annotation_type

annotation_types_points

{21: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 21,
  'name': 'L0',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 22: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 22,
  'name': 'L1',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 23: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 23,
  'name': 'L2',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 24: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'e

In [10]:
annotations = []

image_set = image_sets_api.list_image_sets(name="Multi_IHC", expand="product_set,product_set.annotationtype_set").results[0]

for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id,
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        if anno.annotation_type in annotation_types_points:
            short = Path(image.name).stem.split(" HE")[0]
            
            annotations.append([image.id, image.name, short, anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "image_name_short", "id", "vector", "unique_identifier", "annotation_type"])
annotations

,image_id,image_name,image_name_short,id,vector,unique_identifier,annotation_type
0,334,CRC-A1-1.tif,CRC-A1-1,26236,"{'x1': 25084, 'x2': 25284, 'y1': 16467, 'y2': ...",d68e7b9a-d3ae-43ea-9c7c-4dcac427c588,21
1,334,CRC-A1-1.tif,CRC-A1-1,26237,"{'x1': 25084, 'x2': 25284, 'y1': 45253, 'y2': ...",0ca22054-a283-4010-bde9-ecb8482c1bd3,22
2,334,CRC-A1-1.tif,CRC-A1-1,26238,"{'x1': 25084, 'x2': 25284, 'y1': 74039, 'y2': ...",7fb53235-d051-4d81-9d70-da87725bc6f6,23
3,334,CRC-A1-1.tif,CRC-A1-1,26239,"{'x1': 25084, 'x2': 25284, 'y1': 102825, 'y2':...",2a73c263-0dae-4151-82e0-c661ad26930c,24
4,334,CRC-A1-1.tif,CRC-A1-1,26240,"{'x1': 25084, 'x2': 25284, 'y1': 131611, 'y2':...",0b4c38dd-b8a5-49bb-b0be-735b43e47cac,25
...,...,...,...,...,...,...,...
120,341,CRC-A1-13.tif,CRC-A1-13,26356,"{'x1': 105820, 'x2': 106020, 'y1': 15738, 'y2'...",ce950f3a-6baf-428a-9edc-4fcce2a3f283,41
121,341,CRC-A1-13.tif,CRC-A1-13,26357,"{'x1': 105820, 'x2': 106020, 'y1': 44564, 'y2'...",5986adb0-106d-4598-9d26-e21e78bd4b0e,42
122,341,CRC-A1-13.tif,CRC-A1-13,26358,"{'x1': 105820, 'x2': 106020, 'y1': 73390, 'y2'...",f36851cb-9b29-45f2-98e6-7e75f8cea08e,43
123,341,CRC-A1-13.tif,CRC-A1-13,26359,"{'x1': 105820, 'x2': 106020, 'y1': 102216, 'y2...",9e0f993d-ab04-4de6-886d-6cc2f248a9d6,44


In [11]:
annotations["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations["anno_width"] = [x2-x1 for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["anno_height"]= [y2-y1 for y1, y2 in zip(annotations["y1"], annotations["y2"])]

In [15]:
transformed_annotations_to_upload = []

for image_set_name in tqdm(["Multi_HE"]):
    image_set = image_sets_api.list_image_sets(name=image_set_name, expand="product_set,product_set.annotationtype_set").results[0]

    for image_id in image_set.images:
        
        target_image = images_api.retrieve_image(id=image_id)
        short = Path(target_image.name).stem.split(" HE")[0]
        
        source_annos = annotations[annotations["image_name_short"] == short]
        
        source_name = list(source_annos["image_name"].unique())[0]
        target_name = target_image.name
        
        qtree = pickle.load(open(f'../../ColonCancer/Depth_2/2048/{Path(source_name).stem}-To-{Path(target_name).stem}.pickle', "rb" ))
        #qtree.source_slide = OpenSlide(str(slide_list[source_name]))
        #qtree.target_slide = OpenSlide(str(slide_list[target_name]))
        
        for id, source_anno in source_annos.iterrows():

            box = [source_anno.center_x, source_anno.center_y, source_anno.anno_width, source_anno.anno_height]

            trans_box = qtree.transform_boxes(np.array([box]))[0]

            new_x1 = int(trans_box[0] - trans_box[2] // 2)
            new_y1 = int(trans_box[1] - trans_box[3] // 2)
            new_x2 = int(trans_box[0] + trans_box[2] // 2)
            new_y2 = int(trans_box[1] + trans_box[3] // 2)

            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}

            anno = Annotation(annotation_type=source_anno.annotation_type, 
                                  vector=vector, 
                                  image=target_image.id,
                                 unique_identifier=source_anno.unique_identifier)

            transformed_annotations_to_upload.append(anno)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.24s/it]


In [16]:
transformed_annotations_to_upload[:3]

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 335,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'd68e7b9a-d3ae-43ea-9c7c-4dcac427c588',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 11318, 'x2': 11520, 'y1': 18798, 'y2': 18996},
  'verified_by_user': None},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 335,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': '0ca22054-a283-4010-bde9-ecb8482c1bd3',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 11500, 'x2': 11702, 'y1': 47468, 'y2': 47666},
  'verified_by_user': None},
 {'annotation_type': 23,
  'annotationversion_set': [],
  'blurred': None,
  'concea

In [17]:
#annotations_api.create_annotation(body=transformed_annotations_to_upload)

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 26361,
  'image': 335,
  'last_edit_time': datetime.datetime(2021, 2, 10, 9, 15, 25, 938998),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 10, 9, 15, 25, 938972),
  'unique_identifier': 'd68e7b9a-d3ae-43ea-9c7c-4dcac427c588',
  'uploaded_media_files': [],
  'user': 2,
  'vector': {'x1': 11318, 'x2': 11520, 'y1': 18798, 'y2': 18996},
  'verified_by_user': 'False'},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 26362,
  'image': 335,
  'last_edit_time': datetime.datetime(2021, 2, 10, 9, 15, 25, 939073),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 10, 9, 15, 25, 939058),
  'unique_identifier': '0ca22054-a283-4010-bde9-ecb8482c1bd3',
  'uploaded_media_files': [],
  'user':